In [ ]:
from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
import pandas as pd

ratingsPath = '../ml-latest-small/ratings.csv'
totalUsers = MovieLens.getTotalUsers(MovieLens, ratingsPath)
d1 = {'userId':[],'movieID':[],'recommendedMovieRatng':[]}
df1 = pd.DataFrame(d1)

for x in range (1, 672):
    y = str(x)
    testSubject = y
    k = 10
    
    # Load our data set and compute the user similarity matrix
    ml = MovieLens()
    data = ml.loadMovieLensLatestSmall()
    
    trainSet = data.build_full_trainset()
    
    sim_options = {'name': 'cosine',
                   'user_based': True
                   }
    
    model = KNNBasic(sim_options=sim_options)
    model.fit(trainSet)
    simsMatrix = model.compute_similarities()
    
    # Get top N similar users to our test subject
    # (Alternate approach would be to select users up to some similarity threshold - try it!)
    #testUserInnerID = trainSet(testSubject)
    testUserInnerID = trainSet.to_inner_uid(testSubject)
    similarityRow = simsMatrix[testUserInnerID]
    
    similarUsers = []
    for innerID, score in enumerate(similarityRow):
        if (innerID != testUserInnerID):
            similarUsers.append( (innerID, score) )
    
    kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])
    
    # Get the stuff they rated, and add up ratings for each item, weighted by user similarity
    candidates = defaultdict(float)
    for similarUser in kNeighbors:
        innerID = similarUser[0]
        userSimilarityScore = similarUser[1]
        theirRatings = trainSet.ur[innerID]
        for rating in theirRatings:
            candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore
        
    # Build a dictionary of stuff the user has already seen
    watched = {}
    for itemID, rating in trainSet.ur[testUserInnerID]:
        watched[itemID] = 1
        
    # Get top-rated items from similar users:
    pos = 0
    for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
        if not itemID in watched:
            movieID = trainSet.to_raw_iid(itemID)
            #print("The total number of users are: ", totalUsers)
            #print("The movie recommendation for userId: ", testSubject, " is: ", ml.getMovieName(int(movieID)), " with rating: ", ratingSum, "\n")
            d2 = {'userId':[testSubject],'movieID':[movieID],'recommendedMovieRatng':[ratingSum]}
            df2 = pd.DataFrame(d2)
            pos += 1
            if (pos > 0):
                break
    df1 = df1.append(df2)
print(df1.head())